In [1]:
import os
import requests
import pandas as pd

In [103]:
# move to exceptions.py or errors.py in future
# https://docs.python.org/3/tutorial/errors.html

class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class ResponseError(Error):
    """Exception raised for errors in the API response.

    Attributes:
        message -- explanation of the error
    """

    def __init__(self, message):
        self.message = message

class ApiKeyError(Error):
    """Exception raised for errors in the API response.

    Attributes:
        message -- explanation of the error
    """

    def __init__(self, message):
        self.message = message

In [155]:
# TODO: Move to module at some point
class PhishNetAPI:
    """
    Too lazy rn
    """
    
    def __init__(self):
        """
        Too lazy rn
        """
        self._api_key = os.environ.get('PHISH_API_KEY')
        self._base_url = 'https://api.phish.net/v3'
        self._request_limit = 120
        if self._api_key is None: 
            raise ApiKeyError('No API key found. Check your environment' \
                             'variables and try again.')
        else: 
            self.has_api_key = True
            
    
    # ----------------------- Response Error Handling Utilities -----------------------
    
    def _is_ok_response(self, response):
        """
        Too lazy rn
        """
        if response.status_code == requests.codes.ok: 
            pass
        else: 
            response.raise_for_status()
        
    
    def _response_has_error(self, response):
        """
        Too lazy rn
        """        
        json_response = response.json()
        if json_response.get('error_code') != 0:
            raise ResponseError(f"error code {json_response.get('error_code')}: " \
                                f"{json_response.get('error_message')}")
            
    # ----------------------- Query Parameter Utilities -----------------------

    def _add_api_key_to_query_params(self):
        """
        Too lazy rn
        doc: http://docs.python-requests.org/en/master/user/quickstart/
        """
        payload = {}
        payload['apikey'] = self._api_key
        return payload
            
                                
    def _mask_api_key_from_url(self, response, payload):
        """
        Too lazy rn
        """
        
        url = response.url.replace(payload['apikey'], '<<apikey>>')
                                
        return url
         
                                
    def _mask_api_key_from_query_string(self, response, payload, endpoint):
        """
        Too lazy rn
        """
        
        masked_url = response.url.replace(payload['apikey'], '<<apikey>>')
        query_string = masked_url.replace(endpoint, '')                       
        return query_string
    
    # ----------------------- URL Utilities -----------------------

    def _append_endpoint(self, url, endpoint):
        """
        Too lazy rn
        """
        
        return url + endpoint
    
    
    # ----------------------- Endpoint methods (GET, POST) -----------------------
                                
    def get_all_venues(self):
        """
        Too lazy rn
        """
        
        endpoint = '/venues/all'
        self.endpoint_ = self._append_endpoint(self._base_url, endpoint)
                                
        payload = self._add_api_key_to_query_params()
        response = requests.request("GET", self.endpoint_, params=payload)
                                
        self.url_ = self._mask_api_key_from_url(response, payload)
        self.query_string_ = self._mask_api_key_from_query_string(response, payload, self.endpoint_)
        
        self._is_ok_response(response)
        self._response_has_error(response)
                                
        venues = pd.DataFrame(response.json().get('response').get('data')).transpose()
                                
        return venues
        

In [156]:
phish_api = PhishNetAPI()

In [157]:
venues = phish_api.get_all_venues()

In [158]:
phish_api.has_api_key

True

In [159]:
phish_api.endpoint_

'https://api.phish.net/v3/venues/all'

In [160]:
phish_api.url_

'https://api.phish.net/v3/venues/all?apikey=<<apikey>>'

In [161]:
phish_api.query_string_

'?apikey=<<apikey>>'

In [145]:
venues.head()

,city,country,link,state,venue,venueid
1,Hartford,USA,http://phish.net/venue/1/Meadows_Music_Theatre,CT,Meadows Music Theatre,1
2,Burlington,USA,http://phish.net/venue/2/Nectar%27s,VT,Nectar's,2
3,East Troy,USA,http://phish.net/venue/3/Alpine_Valley_Music_T...,WI,Alpine Valley Music Theatre,3
4,Noblesville,USA,http://phish.net/venue/4/Verizon_Wireless_Musi...,IN,Verizon Wireless Music Center,4
5,Weedsport,USA,http://phish.net/venue/5/Cayuga_County_Fairgro...,NY,Cayuga County Fairgrounds,5


In [162]:
venues.shape

(772, 6)